In [ ]:
%matplotlib widget 

import scipy
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sympy as sp
from itertools import combinations
# from multiprocessing import Pool
from itertools import starmap
import time
import functools
from scipy.interpolate import CloughTocher2DInterpolator
from math import comb

############################
# Setup global variables:  #
############################
n = 4
symbols = sp.symbols('a_:'+str(n**2))
matrix = sp.Matrix(n,n, symbols)

n = 4
symbols = sp.symbols('a_:'+str(n**2))
matrix = sp.Matrix(n,n, symbols)
string_matrix = [['' for i in range(n)] for j in range(n)]

m=0
A = np.zeros((n, comb(n, 2)))
for j in range(n):   
    for k in range(j,n):
        if k == j:
            matrix[j,k] = 'a_16'
            continue
        m+=1
        string_matrix[j][k] = '-a_'+str(m)
        string_matrix[k][j] = '-a_'+str(m)
        matrix[j,k] = 'a_'+str(m)
        matrix[k,j] = 'a_'+str(m)
        A[j][m-1] = 1
        A[k][m-1] = 1

    row_sum = '1'
    for k in range(n):
        row_sum += string_matrix[j][k]
    matrix[j,j] = row_sum

print(A)
print(matrix)


matrix_constraints = tuple(
    {'type': 'eq', 'fun': sp.lambdify([matrix[:]], matrix[k][k], 'numpy')} for k in range (0,n)
)

def sum_kth_principal_minors(matrix, k):
    """Compute the sum of the k-th principal minors of a given matrix."""
    principal_minor_sum = 0
    
    # Get all combinations of row indices for k x k principal minors
    for indices in combinations(range(n), k):
        minor = matrix[indices, indices]  # Extract the k x k principal minor
        principal_minor_sum += minor.det()  # Add the determinant of the minor
    
    return principal_minor_sum

def run_function_with_constraints(loc, constraints = matrix_constraints):
    """Given a function and some constraint functions attempt to minimize the function"""
    bounds = [(0.0, 1.0)] * n**2
    num_starts = 5
    results = []
    
    for _ in range(num_starts):
        initial_guess = np.random.rand(n**2)
        #result = minimize(funcs_of_principal_minors[loc], initial_guess, bounds=bounds, constraints=constraints, method='trust-constr')
        result = minimize(funcs_of_principal_minors[loc], initial_guess, bounds=bounds, constraints=constraints, method='SLSQP')
        results.append(result)
    
    return min(results, key=lambda x: x.fun)

def run_function_with_constraints_and_equal(val, loc, func_num):
    constraints_and_func_1 = matrix_constraints + ({'type': 'eq', 'fun': lambda x : (funcs_of_principal_minors[loc](x) - val)},)
    return run_function_with_constraints(func_num, constraints_and_func_1)

def run_function_with_constraints_and_double_equal(val_1, val_2, loc_1, loc_2, func_num):
    constraints_and_func_2 = matrix_constraints + ({'type': 'eq', 'fun': lambda x : (funcs_of_principal_minors[loc_1](x) - val_1)},) + ({'type': 'eq', 'fun': lambda x : (funcs_of_principal_minors[loc_2](x) - val_2)},)
    return run_function_with_constraints(func_num, constraints_and_func_2)

funcs_of_principal_minors = tuple(
    sp.lambdify([matrix[:]], sum_kth_principal_minors(matrix, k), 'numpy')
    for k in range(1, n+1)
) + tuple(
    sp.lambdify([matrix[:]], -1*sum_kth_principal_minors(matrix, k), 'numpy')
    for k in range(1, n+1)
)
